In [1]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
import mxnet as mx
import numpy as np

from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import os
import sys
import math
import time
import pickle

proj_dir = os.getcwd()
label_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/labels')
lld_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/lld')

pickle_train_y_list = 'pickle_train_y_list'
pickle_train_x_list = 'pickle_train_x_list'

pickle_test_y_list = 'pickle_test_y_list'
pickle_test_x_list = 'pickle_test_x_list'

In [3]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

In [4]:
def calc_score(test_y_list, predict_y_list):
    print "Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):"
    print 'macro   : ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='macro')
    print 'weighted: ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='weighted')

In [5]:
# z-score normalization
scaler = preprocessing.StandardScaler().fit(train_x_list)
train_x_list = scaler.transform(train_x_list)
test_x_list = scaler.transform(test_x_list)

In [6]:
# PCA whiten
# pca = PCA(n_components='mle', svd_solver='full', whiten=True)
# x_pca = pca.fit(train_x_list)
# train_x_list = pca.transform(train_x_list)
# test_x_list = pca.transform(test_x_list)

In [7]:
# sklearn l1/l2 normalization
# x_normalizer = preprocessing.Normalizer(norm='l2').fit(train_x_list)
# train_x_list = x_normalizer.transform(train_x_list)
# test_x_list = x_normalizer.transform(test_x_list)

In [8]:
# # SMOTE up-sampling training data
# sm = SMOTE(kind='regular')
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)

# # split training data to 5 different classes
# train_x_lists = [[] for i in range(5)]
# for i in xrange(len(train_y_list)):
#     train_x_lists[int(train_y_list[i])].append(train_x_list[i])
# for i in xrange(5):
#     print 'number of class', i, ':', len(train_x_lists[i])

In [9]:
print train_y_list.shape, train_x_list.shape
print test_y_list.shape, test_x_list.shape

(9959,) (9959, 384)
(8257,) (8257, 384)


In [10]:
print train_x_list[0][:10]
print test_x_list[0][:10]

[-1.28200357  0.06173483 -1.28187449 -0.41449729  0.46579774 -1.17798747
  0.06471169 -0.85753636 -0.64435287 -1.27747438]
[-1.13903374 -0.75002097 -1.13420654 -0.43244806 -0.38227578 -1.17669163
  0.04549002 -0.83947808 -0.63455673 -1.16125476]


In [11]:
X = train_x_list
y = train_y_list

In [12]:
from sklearn_extensions.extreme_learning_machines.elm import GenELMClassifier
from sklearn_extensions.extreme_learning_machines.random_layer import RBFRandomLayer, MLPRandomLayer


def make_classifiers():
    names = [
        "ELM(tanh)",
        "ELM(sinsq)",
        "ELM(tribas)",
        "ELM(hardlim)",
        "ELM(rbf(0.1))"
    ]

    nh = 1000

    # pass user defined transfer func
    sinsq = (lambda x: np.power(np.sin(x), 2.0))
    srhl_sinsq = MLPRandomLayer(n_hidden=nh, activation_func=sinsq)

    # use internal transfer funcs
    srhl_tanh = MLPRandomLayer(n_hidden=nh, activation_func='tanh')
    srhl_tribas = MLPRandomLayer(n_hidden=nh, activation_func='tribas')
    srhl_hardlim = MLPRandomLayer(n_hidden=nh, activation_func='hardlim')

    # use gaussian RBF
    srhl_rbf = RBFRandomLayer(n_hidden=nh*2, rbf_width=0.1, random_state=0)

    classifiers = [
        GenELMClassifier(hidden_layer=srhl_tanh),
        GenELMClassifier(hidden_layer=srhl_sinsq),
        GenELMClassifier(hidden_layer=srhl_tribas),
        GenELMClassifier(hidden_layer=srhl_hardlim),
        GenELMClassifier(hidden_layer=srhl_rbf)
    ]

    return names, classifiers

names, classifiers = make_classifiers()

# iterate over classifiers
for name, clf in zip(names, classifiers):
    print "classifier name: ", name
    clf.fit(X, y)
    predict_y_list = clf.predict(test_x_list)
    calc_score(test_y_list, predict_y_list)

classifier name:  ELM(tanh)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.316977036019228, 0.29536223266563211, 0.2978024527913618, None)
weighted:  (0.55953943371506709, 0.60469904323604218, 0.57725427956755948, None)
classifier name:  ELM(sinsq)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.19183466965250645, 0.19946539306383027, 0.17805009810098213, None)
weighted:  (0.46602891434338001, 0.60021799685115662, 0.51465413112937997, None)
classifier name:  ELM(tribas)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.20625406873120961, 0.20168320073488261, 0.18488255978482021, None)
weighted:  (0.47036908042867076, 0.59597916918977834, 0.51504413226569545, None)
classifier name:  ELM(hardlim)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.35572729512571999, 0.28383948197696307, 0.29273966396899465, None)
weighted:  (0.5702524207539682, 0.64151628920915582, 0.59142703681231135, None)
classifier name:  

In [13]:
# SMOTE up-sampling training data
sm = SMOTE(kind='regular')
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list, train_y_list)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list)):
    train_x_lists[int(train_y_list[i])].append(train_x_list[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

No handlers could be found for logger "imblearn.base"


number of class 0 : 881
number of class 1 : 2093
number of class 2 : 5590
number of class 3 : 674
number of class 4 : 721


In [14]:
from sklearn_extensions.extreme_learning_machines.elm import GenELMClassifier
from sklearn_extensions.extreme_learning_machines.random_layer import RBFRandomLayer, MLPRandomLayer


def make_classifiers():
    names = [
        "ELM(tanh)",
        "ELM(sinsq)",
        "ELM(tribas)",
        "ELM(hardlim)",
        "ELM(rbf(0.1))"
    ]

    nh = 1000

    # pass user defined transfer func
    sinsq = (lambda x: np.power(np.sin(x), 2.0))
    srhl_sinsq = MLPRandomLayer(n_hidden=nh, activation_func=sinsq)

    # use internal transfer funcs
    srhl_tanh = MLPRandomLayer(n_hidden=nh, activation_func='tanh')
    srhl_tribas = MLPRandomLayer(n_hidden=nh, activation_func='tribas')
    srhl_hardlim = MLPRandomLayer(n_hidden=nh, activation_func='hardlim')

    # use gaussian RBF
    srhl_rbf = RBFRandomLayer(n_hidden=nh*2, rbf_width=0.1, random_state=0)

    classifiers = [
        GenELMClassifier(hidden_layer=srhl_tanh),
        GenELMClassifier(hidden_layer=srhl_sinsq),
        GenELMClassifier(hidden_layer=srhl_tribas),
        GenELMClassifier(hidden_layer=srhl_hardlim),
        GenELMClassifier(hidden_layer=srhl_rbf)
    ]

    return names, classifiers

names, classifiers = make_classifiers()

# iterate over classifiers
for name, clf in zip(names, classifiers):
    print "classifier name: ", name
    clf.fit(X, y)
    predict_y_list = clf.predict(test_x_list)
    calc_score(test_y_list, predict_y_list)

classifier name:  ELM(tanh)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.31380803308220451, 0.29310870056840338, 0.29755179732153447, None)
weighted:  (0.563884493599489, 0.61087562068547896, 0.5832389757120725, None)
classifier name:  ELM(sinsq)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.21081486135025154, 0.20001766611769328, 0.17908888750690616, None)
weighted:  (0.4725776402444683, 0.60227685600096892, 0.51528825822503233, None)
classifier name:  ELM(tribas)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.21433375109951239, 0.20311560000763232, 0.186145986655114, None)
weighted:  (0.47513654791125015, 0.59888579387186625, 0.51761738503294497, None)
classifier name:  ELM(hardlim)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.32821150349179329, 0.27313415043369899, 0.27727319473825196, None)
weighted:  (0.55975630758695039, 0.63546082112147273, 0.58508803229954742, None)
classifier name:  EL

In [15]:
print predict_y_list[:100]

[2 3 2 2 2 0 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 3 0 1 4 2 4 2 2 1 2 2 2 2 2 2 1
 2 4 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2
 1 2 2 2 1 1 1 1 2 2 2 1 1 1 1 1 2 2 2 2 2 2 2 1 1 1]


In [16]:
print test_y_list[:100]

[2 2 2 2 2 4 2 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1
 1 2 2 2 2 2 2 2 1 1 2 1 1 2 2 2 0 1 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2
 1 1 1 2 1 1 1 1 1 2 2 1 1 1 2 1 1 2 2 2 2 2 1 1 1 2]
